In [3]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTWalker import GPTWalkerEnv

In [20]:

folder_name = "results/Eureka_morphology"
log_file = os.path.join(folder_name, "parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

iterations = 5
morphology_nums = 16
rewardfunc_nums = 1

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []


In [ ]:
import prompts
class DGA:
    def __init__(self):
        api_key = "<api_key>"
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4-turbo"


    def generate_morphology_eureka(self, morphology_nums, best_message, folder_name, iteration):

        messages = [
                    {"role": "system", "content": "You are a helpful mujoco robot designer"},
                    {"role": "user", "content": morphology_prompts + best_message + morphology_format}
                    ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums
        )

        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_walker_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = ant_design(parameter)  
            filename = f"GPTWalker_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
        


In [1]:
eureka_morphology_prompts = """Role: You are a robot designer trying to design robot parameters to increase the fitness function as effective as possible.
Task: Your task is to design parameters of robot that will help agent achieve the fitness function as high as possible.
fintess function: walk distance/material cost.
Description: The walker is a two-dimensional bipedal robot consisting of seven main body parts - a single torso at the top (with the two legs splitting after the torso), two thighs in the middle below the torso, two legs below the thighs, and two feet attached to the legs on which the entire body rests. 
The goal is to walk in the forward (right) direction by applying torque to the six hinges connecting the seven body parts., you should design parameter to make the robot move as faster as possible.

Here is the xml file:
  <mujoco model="walker2d">
    <compiler angle="degree" coordinate="global" inertiafromgeom="true"/>
    <default>
      <joint armature="0.01" damping=".1" limited="true"/>
      <geom conaffinity="0" condim="3" contype="1" density="1000" friction=".7 .1 .1" rgba="0.8 0.6 .4 1"/>
    </default>
    <option integrator="RK4" timestep="0.002"/>

    <worldbody>
      <light cutoff="100" diffuse="1 1 1" dir="-0 0 -1.3" directional="true" exponent="1" pos="0 0 1.3" specular=".1 .1 .1"/>
      <geom conaffinity="1" condim="3" name="floor" pos="0 0 0" rgba="0.8 0.9 0.8 1" size="20 20 .125" type="plane" material="MatPlane"/>

      <body name="torso">
        <camera name="track" mode="trackcom" pos="0 -3 -0.25" xyaxes="1 0 0 0 0 1"/>
        <joint armature="0" axis="1 0 0" damping="0" limited="false" name="ignore1" pos="0 0 0" stiffness="0" type="slide"/>
        <joint armature="0" axis="0 0 1" damping="0" limited="false" name="ignore2" pos="0 0 0" ref="1.25" stiffness="0" type="slide"/>
        <joint armature="0" axis="0 1 0" damping="0" limited="false" name="ignore3" pos="0 0 0" stiffness="0" type="hinge"/>
        <geom fromto="0 0 {param1} 0 0 {param2}" name="torso_geom" size="{param8}" type="capsule" friction="0.9"/>
        <body name="thigh">
          <joint axis="0 -1 0" name="thigh_joint" pos="0 0 {param2}" range="-150 0" type="hinge"/>
          <geom fromto="0 0 {param2} 0 0 {param3}" name="thigh_geom" size="{param8}" type="capsule" friction="0.9"/>
          <body name="leg">
            <joint axis="0 -1 0" name="leg_joint" pos="0 0 {param3}" range="-150 0" type="hinge"/>
            <geom fromto="0 0 {param3} 0 0 {param4}" name="leg_geom" size="{param9}" type="capsule" friction="0.9"/>
            <body name="foot">
              <joint axis="0 -1 0" name="foot_joint" pos="0 0 {param4}" range="-45 45" type="hinge"/>
              <geom fromto="{param5} 0 {param4} {param6} 0 {param4}" name="foot_geom" size="{param10}" type="capsule" friction="1.9"/>
            </body>
          </body>
        </body>
        <body name="thigh_left">
          <joint axis="0 -1 0" name="thigh_left_joint" pos="0 0 {param2}" range="-150 0" type="hinge"/>
          <geom fromto="0 0 {param2} 0 0 {param3}" name="thigh_left_geom" size="{param8}" type="capsule" friction="0.9"/>
          <body name="leg_left">
            <joint axis="0 -1 0" name="leg_left_joint" pos="0 0 {param3}" range="-150 0" type="hinge"/>
            <geom fromto="0 0 {param3} 0 0 {param4}" name="leg_left_geom" size="{param9}" type="capsule" friction="0.9"/>
            <body name="foot_left">
              <joint axis="0 -1 0" name="foot_left_joint" pos="0 0 {param4}" range="-45 45" type="hinge"/>
              <geom fromto="{param5} 0 {param4} {param6} 0 {param4}" name="foot_left_geom" size="{param10}" type="capsule" friction="1.9"/>
            </body>
          </body>
        </body>
      </body>
    </worldbody>

    <actuator>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="100" joint="thigh_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="100" joint="leg_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="100" joint="foot_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="100" joint="thigh_left_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="100" joint="leg_left_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="100" joint="foot_left_joint"/>
    </actuator>
      <asset>
        <texture type="skybox" builtin="gradient" rgb1=".4 .5 .6" rgb2="0 0 0"
            width="100" height="100"/>
        <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
        <texture builtin="checker" height="100" name="texplane" rgb1="0 0 0" rgb2="0.8 0.8 0.8" type="2d" width="100"/>
        <material name="MatPlane" reflectance="0.5" shininess="1" specular="1" texrepeat="60 60" texture="texplane"/>
        <material name="geom" texture="texgeom" texuniform="true"/>
      </asset>
  </mujoco>

1. For reducing material cost to ensure the effieciency of robot design, you should reduce redundant paramters (e.g., smaller geom size) and increase paramters who control the robot (e.g., longer leags, ankles).
2. Your design should fit the control gear and others parts of robots well.

There are also some design parameters and their fitness. 
Please carefully observe these parameters and their fitness, try to design a new parameter to further improve the fitness.

"""

In [ ]:
designer = DGA()
rewardfunc_list = [f'results/Eureka_morphology/env/GPTrewardfunc_{i}.py' for i in range(0,1)]
best_message = ""
for iter in range(iterations):
    morphology_list, material_list, parameter_list = designer.generate_morphology_eureka(morphology_nums, best_message, folder_name, iteration=iter)
    for i, rewardfunc in enumerate(rewardfunc_list):
        for j, morphology in enumerate(morphology_list):

            print(i, rewardfunc)
            print(j, morphology)
            shutil.copy(morphology, "GPTWalker.xml")
            shutil.copy(rewardfunc, "GPTrewardfunc.py")         

            import GPTrewardfunc
            importlib.reload(GPTrewardfunc)  # 重新加载模块
            from GPTrewardfunc import _get_rew
            GPTWalkerEnv._get_rew = _get_rew

            env_name = "GPTWalkerEnv"
            model_path = Train(j,  i, folder_name, total_timesteps=5e5)
            fitness, reward = Eva(model_path)

            material = material_list[j]
            efficiency = fitness/material
            fitness_matrix[i][j] = fitness
            efficiency_matrix[i][j] = efficiency
            
            logging.info("___________________finish coarse optimization_____________________")
            logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

    best_index = np.argmax(efficiency_matrix)
    best_efficiency = np.max(efficiency_matrix[:][0])
    best_parameter = parameter_list[best_index]
    best_message = eureka_morphology_prompts + f"\n best parameter:{best_parameter} \n" + f"best fintess:{best_efficiency}" 